In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import ObjectiveFunction as of
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
import torch
import helper_funcs as hf
from math import sqrt
from scipy.stats import uniform, norm, levy_stable
from levy.__init__ import fit_levy
import itertools

In [8]:
# Testing hypersphere sample
def hypersphere_sample(dim):

    direction = np.random.normal(size=dim)
    direction = np.sqrt(sum(direction**2))**(-1) * direction

    return torch.from_numpy(direction)

samples = []
for i in range(1000):
    direction = hypersphere_sample(2)
    samples.append(direction)
samples = hf.vectors_to_nested_list(samples)
px.scatter(x=samples[0], y=samples[1], width=800, height=800)

In [14]:
samples = []
for i in range(1000):
    direction = hypersphere_sample(1)
    samples.append(float(direction))
px.scatter(x=samples, width=800, height=800)

In [10]:
def alpha_estimator(m, X):
    # X is tensor with shape Nxd
    N = len(X)
    n = int(N/m)
    Y = torch.sum(X.view(n, m,-1), 1)
    eps = np.spacing(1)
    Y_log_norm = torch.log(Y.norm(dim=1) + eps).mean()
    X_log_norm = torch.log(X.norm(dim=1) + eps).mean()
    diff = (Y_log_norm - X_log_norm) / np.log(m)
    return 1 / diff

def alpha_log_moment_estimator(X):
    # Problem ---> does not do well with low sample size (slow estimator convergence)
    gamma = np.euler_gamma
    denom = gamma + torch.mean(torch.log(torch.abs(X)))
    return (gamma/denom).item()

# Corollary 2.4 in Mohammadi 2014
def mohammadi1_alpha_estimator(m, X):
    N = len(X)
    n = int(N/m)
    Y = torch.sum(X.view(n, m, -1), 1)
    eps = np.spacing(1)
    # Y_log_norm = torch.log(torch.linalg.norm(Y, dim=1) + eps).mean()
    # X_log_norm = torch.log(torch.linalg.norm(Y, dim=1) + eps).mean()
    Y_log_norm = torch.log(Y.abs() + eps).mean()
    X_log_norm = torch.log(X.abs() + eps).mean()
    diff = (Y_log_norm - X_log_norm) / np.log(m)
    return (1 / diff).item()

N = 200
X = levy_stable.rvs(1.5, 0, scale=1, size=N)
for i in range(1+int(np.sqrt(N)), 1, -1):
    if N % i == 0:
        m = i
        break
X = torch.from_numpy(X)
a1 = mohammadi1_alpha_estimator(m,X)
a2 = alpha_log_moment_estimator(X)
a3 = fit_levy(np.array(X), beta=0.0, mu=0.0)[0].get()[0]
print(a1, a2, a3)
print(fit_levy(np.array(X), beta=0.0, mu=0.0)[0])





1.5452564484784197 1.3791046752312273 1.4361059229047328
alpha: 1.44, beta: 0.00, mu: 0.00, sigma: 0.94. Parametrization: 0.


In [197]:
n = int(N/m)
Y = torch.sum(X.view(n,m,-1), 1)
eps = np.spacing(1)
Ylog_norm = torch.log(torch.linalg.norm(Y, dim=1) + eps).mean()
Xlog_norm = torch.log(torch.linalg.norm(X, dim=1) + eps).mean()
print(Ylog_norm, Xlog_norm)
diff = (Ylog_norm - Xlog_norm) / np.log(m)


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [174]:
a1s, a2s, a3s= [], [], []
N = 6000
for i in range(1+int(np.sqrt(N)), 1, -1):
    if N % i == 0:
        m = i
        break

for i in range(1000):
    X = levy_stable.rvs(1.7, 0, scale=1, size=N)
    X = torch.from_numpy(X)
    a1 = mohammadi1_alpha_estimator(m,X)
    a2 = alpha_log_moment_estimator(X)
    a3 = fit_levy(np.array(X), beta=0.0, mu=0.0)[0].get()[0]
    a1s.append(a1)
    a2s.append(a2)
    a3s.append(a3)


In [175]:
px.histogram(a1s)


In [176]:
px.histogram(a2s)

In [177]:
px.histogram(a3s)

In [ ]:
from ObjectiveFunction import Hodgkinson
start = torch.tensor([2.5, 2.0])
data = [[0,0], [1,1], [0, 1]]
bounds = 20
f = Hodgkinson(start,data,bounds=bounds)

xlim, ylim, res = (-10, 10), (-10, 10), 0.1
X = np.arange(xlim[0], xlim[1], res)
Y = np.arange(ylim[0], ylim[1], res)
grid = [np.array(_) for _ in itertools.product(X,Y)]
Z = [float(f.forward(torch.Tensor(_))) for _ in grid]
Z = np.array(Z).reshape(len(X), len(Y)).T

fig = go.Figure()
fig.add_trace(
    go.Contour(x=X, y=Y, z=Z)
)
fig.update_layout(
    width=800,
    height=800
)

In [22]:
scale = 0.05
levy_r = levy_stable.rvs(1.1, 0, scale=scale, size=1000)
levy_r = np.abs(levy_r)
px.histogram(levy_r, nbins=500)
